In [ ]:
!pip install mlflow
import numpy as np
import pandas as pd
import mlflow

In [ ]:
# Cargar datos
datos = pd.read_csv("datos_lineales.csv")
x = datos["x"].values
y = datos["y"].values

# Añadir columna de unos para el término constante
X = np.vstack((np.ones(len(x)), x)).T
N = len(y)

# Parámetros
NITMAX = 1000
gamma_valor = 0.01


In [ ]:
# -------------------------
# Descenso de gradiente
# -------------------------
def gradiente(X, y, NITMAX, gamma):
    theta = np.zeros(X.shape[1])
    for _ in range(NITMAX):
        y_pred = np.dot(X, theta)
        grad = (1 / len(y)) * np.dot(X.T, (y_pred - y))
        theta = theta - gamma * grad
    return theta

# ---- Run 1: Gradiente ----
with mlflow.start_run(run_name="gradiente"):
    theta_gd = gradiente(X, y, NITMAX, gamma_valor)
    mse_gd = np.mean((X @ theta_gd - y)**2)

    mlflow.log_param("algorithm", "gradiente")
    mlflow.log_param("gamma", gamma_valor)
    mlflow.log_param("NITMAX", NITMAX)
    mlflow.log_metric("mse", mse_gd)
    np.save("theta_gd.npy", theta_gd)
    mlflow.log_artifact("theta_gd.npy")

# -------------------------
# Descenso de gradiente estocástico
# -------------------------
def sgd(X, y, NITMAX, gamma):
    theta = np.zeros(X.shape[1])
    for j in range(NITMAX):
        for i in range(len(y)):
            xi = X[i, :].reshape(1, -1)
            yi = y[i]
            y_pred = np.dot(xi, theta)
            grad = (y_pred - yi) * xi
            theta = theta - gamma * grad.squeeze()
    return theta

# ---- Run 2: Estocástico ----
with mlflow.start_run(run_name="sgd"):
    theta_sgd = sgd(X, y, NITMAX, gamma_valor)
    mse_sgd = np.mean((X @ theta_sgd - y)**2)

    mlflow.log_param("algorithm", "sgd")
    mlflow.log_param("gamma", gamma_valor)
    mlflow.log_param("NITMAX", NITMAX)
    mlflow.log_metric("mse", mse_sgd)
    np.save("theta_sgd.npy", theta_sgd)
    mlflow.log_artifact("theta_sgd.npy")

# -------------------------
# Mostrar resultados
# -------------------------
print("Coeficientes GD:", theta_gd)
print("Coeficientes SGD:", theta_sgd)
print("MSE GD:", mse_gd)
print("MSE SGD:", mse_sgd)